In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[6],4
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -9.077911e-06                    1 -0.000020
0.000750        2 -9.044177e-06                    2 -0.000042
0.001052        3 -8.977873e-06                    3 -0.000008
0.001476        4 -8.891137e-06                    4 -0.000008
0.002070        5 -8.802642e-06                    5 -0.000007
0.002904        6 -8.701331e-06                    6 -0.000008
0.004074        7 -8.587891e-06                    7 -0.000007
0.005714        8 -8.467483e-06                    8 -0.000007
0.008015        9 -8.334229e-06                    9 -0.000007
0.011243       10 -8.189517e-06                   10 -0.000007
0.015771       11 -8.033162e-06                   11 -0.000007
0.022122       12 -7.864161e-06                   12 -0.000007
0.031031       13 -7.681932e-06                   13 -0.000008
0.043528       14 -7.429187e-06                   14 -0.000012
0.061057       15 -6.352139e-06                   15 -0.000012
0.085645       16 -3.143961e-06                   16 -0.000008
0.120136       17  3.070876e-06                   17 -0.000005
0.168516       18  1.316197e-05                   18 -0.000001
0.236378       19  2.882620e-05                   19  0.000005
0.331549       20  5.243065e-05                   20  0.000014
0.465100       21  8.718533e-05                   21  0.000028
0.652400       22  1.317870e-04                   22  0.000048
0.915100       23  1.501447e-04                   23  0.000056
1.283650       24  1.322895e-04                   24  0.000048
1.800600       25  1.043212e-04                   25  0.000037
2.525700       26  7.352555e-05                   26  0.000024
3.542800       27  4.864666e-05                   27  0.000014
4.969550       28  3.025854e-05                   28  0.000006
6.970850       29  1.688221e-05                   29  0.000001
9.778100       30  7.295138e-06                   30 -0.000003
13.715850      31  5.647570e-07                   31 -0.000005
19.239350      32 -4.019626e-06                   32 -0.000007
26.987250      33 -7.044272e-06                   33 -0.000008
37.855300      34 -8.977668e-06                   34 -0.000008
53.100050      35 -1.019125e-05                   35 -0.000009
73.887500      36 -1.031378e-05                   36 -0.000009
97.662500      37 -1.005772e-05                   37 -0.000009
121.437500     38 -1.066284e-05                   38 -0.000009
145.212500     39 -1.178797e-05                   39 -0.000011
168.987500     40 -1.345305e-05                   40 -0.000013
192.762500     41 -1.895580e-05                   41 -0.000020
216.537500     42 -3.707884e-05                   42 -0.000046
240.312500     43 -5.249580e-05                   43 -0.000084
264.087500     44 -3.810979e-05                   44 -0.000126
287.862500     45  6.037806e-05                   45 -0.000141
311.637500     46  3.305724e-04                   46 -0.000071
335.412500     47  8.930705e-04                   47  0.000179
359.187500     48  1.869639e-03                   48  0.000731
382.962500     49  3.218443e-03                   49  0.001649
406.737500     50  5.118575e-03                   50  0.003071
430.512500     51  7.664960e-03                   51  0.005113
454.287500     52  1.093965e-02                   52  0.007935
478.062500     53  1.501085e-02                   53  0.011681
501.837500     54  1.989912e-02                   54  0.016414
525.612500     55  2.567105e-02                   55  0.022040
549.387500     56  3.232143e-02                   56  0.028663
573.162500     57  3.987959e-02                   57  0.036270
596.937500     58  4.840540e-02                   58  0.044803
620.712500     59  5.789584e-02                   59  0.054156
644.487500     60  6.843980e-02                   60  0.064153
668.262500     61  7.993459e-02              

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -65.454986  0.000000e+00 -65.454986 -65.458520   
0.000624    2     -65.454986  5.260138e-10 -65.454986 -65.458520   
0.000876    3     -65.454986  7.414555e-10 -65.454986 -65.458520   
0.001229    4     -65.454986  1.054060e-09 -65.454986 -65.458520   
0.001723    5     -65.454986  1.508019e-09 -65.454986 -65.458520   
0.002417    6     -65.454986  2.167135e-09 -65.454986 -65.458520   
0.003391    7     -65.454986  3.124159e-09 -65.454986 -65.458520   
0.004757    8     -65.454986  4.513637e-09 -65.454986 -65.458520   
0.006672    9     -65.454986  6.530760e-09 -65.454986 -65.458520   
0.009359    10    -65.454986  9.458924e-09 -65.454986 -65.458520   
0.013128    11    -65.454986  1.370930e-08 -65.454986 -65.458520   
0.018415    12    -65.454986  1.987869e-08 -65.454986 -65.458520   
0.025830    13    -65.454986  2.883298e-08 -65.454986 -65.458520   
0.036232    14    -65.454986  4.183025e-08 -65.454986 -65.458520   
0.050823    15    -65.454986  6.087255e-08 -65.454986 -65.458520   
0.071291    16    -65.454986  9.196099e-08 -65.454986 -65.458520   
0.100000    17    -65.454986  1.506978e-07 -65.454986 -65.458520   
0.140271    18    -65.454986  2.669430e-07 -65.454986 -65.458520   
0.196760    19    -65.454986  4.964204e-07 -65.454986 -65.458520   
0.275997    20    -65.454987  9.476689e-07 -65.454986 -65.458521   
0.387100    21    -65.454987  1.831155e-06 -65.454985 -65.458521   
0.543100    22    -65.454987  3.555890e-06 -65.454983 -65.458521   
0.761700    23    -65.454987  6.798860e-06 -65.454980 -65.458521   
1.068500    24    -65.454986  1.181466e-05 -65.454975 -65.458522   
1.498800    25    -65.454986  1.821750e-05 -65.454968 -65.458522   
2.102400    26    -65.454986  2.578379e-05 -65.454960 -65.458523   
2.949000    27    -65.454987  3.413795e-05 -65.454953 -65.458524   
4.136600    28    -65.454989  4.318214e-05 -65.454946 -65.458527   
5.802500    29    -65.454993  5.294263e-05 -65.454940 -65.458530   
8.139200    30    -65.454998  6.345100e-05 -65.454935 -65.458535   
11.417000   31    -65.455007  7.474026e-05 -65.454932 -65.458541   
16.014700   32    -65.455019  8.684349e-05 -65.454932 -65.458550   
22.464000   33    -65.455035  9.981135e-05 -65.454935 -65.458562   
31.510500   34    -65.455056  1.136860e-04 -65.454943 -65.458577   
44.200100   35    -65.455084  1.283847e-04 -65.454956 -65.458597   
62.000000   36    -65.455120  1.424350e-04 -65.454978 -65.458623   
85.775000   37    -65.455161  1.539974e-04 -65.455006 -65.458655   
109.550000  38    -65.455197  1.626842e-04 -65.455034 -65.458685   
133.325000  39    -65.455238  1.734095e-04 -65.455065 -65.458719   
157.100000  40    -65.455289  1.908696e-04 -65.455098 -65.458761   
180.875000  41    -65.455357  2.201850e-04 -65.455136 -65.458816   
204.650000  42    -65.455472  2.824903e-04 -65.455190 -65.458912   
228.425000  43    -65.455772  4.789657e-04 -65.455294 -65.459170   
252.200000  44    -65.456445  1.003094e-03 -65.455441 -65.459762   
275.975000  45    -65.457777  2.228263e-03 -65.455549 -65.460957   
299.750000  46    -65.460180  4.800476e-03 -65.455379 -65.463145   
323.525000  47    -65.464186  9.737766e-03 -65.454448 -65.466848   
347.300000  48    -65.470453  1.851952e-02 -65.451932 -65.472673   
371.075000  49    -65.479623  3.295547e-02 -65.446667 -65.481270   
394.850000  50    -65.491700  5.409713e-02 -65.437603 -65.492718   
418.625000  51    -65.507188  8.399987e-02 -65.423188 -65.507508   
442.400000  52    -65.526583  1.249823e-01 -65.401600 -65.526082   
466.175000  53    -65.550368  1.795773e-01 -65.370791 -65.548995   
489.950000  54    -65.579001  2.504854e-01 -65.328516 -65.576777   
513.725000  55    -65.612838  3.403644e-01 -65.272473 -65.609875   
537.500000  56    -65.652225  4.520492e-01 -65.200176 -65.648401   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')